# やること

## パラメータは？

パラメータとして与えるのは以下

①　$\rho_{\chi_{k}}$　:　 $\chi_{k}$に対するbeliefのprior precision。　candidateごとに用意する。

②　$\mu_{\chi_{k}}$　：　$\chi_{k}$に対するbeliefのmean。candidateごとに用意する。

③　$\rho_{\eta}$　：　シグナルのprecision。一つで良い。

④　$\chi_{k}$　：　candidate kのtrue quality。candidateごとに用意する。

## belief updating

③と④をgivenとして$s_{1,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

1期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta}s_{1, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

このposterior meanを$E\left[\chi_{k}|\Omega_{1}\right]$として記録する。

以降t期においても同様に

③と④をgivenとして$s_{t,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

t期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta} \sum_{\tau = 1}^{t} s_{\tau, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

そしてそのposterior meanを$E\left[\chi_{k}|\Omega_{t}\right]$として記録する。

## 詳細

パラメータセット一つにつきT回シグナルを乱数発生する。使うパラメータはいつも一緒なので出てくる数字に大差はない。

違うのはその出てくる順番。T!順あり得るので一つのパラメータセットにつきT!回1期からT期までの$E\left[\chi_{k}|\Omega_{t}\right]$を記録する。

さらにパラメータは100セット用意。

以下では、candidate4人について$E\left[\chi_{k}|\Omega_{t}\right]$のpathを100×T!通り作成するためのclassを作成する。

In [14]:
import numpy as np
from scipy.special import factorial

In [15]:
class ExpectedChi:
    
    def __init__(self, lam, T, sim):
        self.lam = lam # parameter of exponential distribution which rho's follow
        self.T = T # time
        self.sim = sim # simulation number
    
    def setparam(self):
        # params[0, :]　：　rho_eta
        # params[(1,2,3,4), :] : rho_chi for each candidate
        # params[(5,6,7,8), :] : mu_chi for each candidate
        # params[(9,10,11,12), :] : chi for each candidate
        rho = np.random.exponential(self.lam, (5, self.sim))
        mu_chi = np.random.normal(0.0,1.0,(8, self.sim))
        params = np.vstack((rho, mu_chi))
        return params
        
    def signals(self):
        # make the 4 dimentional array : all_signals
        # each element in all_signals is 3 dimentional array whose shape is (T! * 4 * T )
        params = self.setparam()
        all_signals = np.empty([self.sim, factorial(self.T).astype(int), 4, self.T])
        for i in range(self.sim):
            signals_differentparam = np.empty([factorial(self.T).astype(int), 4, self.T])
            all_signals[i, :, :, :] = signals_differentparam
            for k in range(factorial(self.T).astype(int)):
                signals = np.empty([4, self.T])
                signals_differentparam[k, :, :] = signals
                for j in range(4):
                    signal = np.random.normal(params[j + 9, i], 1/params[0, i],  (1, self.T))
                    signals[j, :] = signal
        return all_signals
    
    def cum_signals(self):
        # calculating cumulative signals for expected chi_k given omega
        sig = self.signals()
        for i in range(self.sim):
            for j in range(self.T):
                if j != 0:
                    sig[i, :, :, :][:, :, j] = sig[i, :, :, :][:, :, j] + sig[i, :, :, :][:, :, j -1]
        return sig
            
    def expected_chi(self):
        # calculating expected chi_k given omega
        cum = self.cum_signals()
        params = self.setparam()
        transform = np.empty((self.sim, factorial(self.T).astype(int), 4, self.T))
        
        # first multiply cum by rho_eta 
        for i in range(self.sim):
            transform[i, :, :, :] = cum[i, :, :, :] * params[0, i]
            
        # second add rho_chi + mu_chi to transform
        for i in range(self.sim):
            for j in range(4):
                transform[i, :, j, :] = transform[i, :, j, :] + (params[j + 1, i] * params[j + 5, i])
        
        # third divide by rho_chi + t * rho_eta
        for i in range(self.sim):
            for j in range(4):
                for t in range(self.T):
                    transform[i, :, j, t] = transform[i, :, j, t] / (params[j + 1, i] + t * params[0, i])
        
        return transform
    
    def means(self, each_sim = True):
        # calculate mean
        if each_sim == True:
            # calculate the mean of expected chi for each parameter set
            # return the list whose length is self.sim that contains the means of each parameter set
            return self.expected_chi().mean(axis = (1,2,3))
        
        else:
            # calcualte the over all mean
            return self.expected_chi().mean()
        
    def variances(self, each_sim = True):
        # calculate variance
        if each_sim == True:
            # calculate the variances of expected chi for each parameter set
            # return the list whose length is self.sim that contains the variances of each parameter set
            return self.expected_chi().var(axis = (1,2,3))
        
        else:
            # calcualte the over all variances
            return self.expected_chi().var()

In [16]:
test = ExpectedChi(lam = 1.0, T = 3, sim = 10)

In [17]:
test.setparam()

array([[  2.31488484e-01,   1.45064405e-01,   1.15508030e+00,
          6.46763526e+00,   1.15701099e+00,   4.48055753e-01,
          8.72289089e-01,   4.47868847e-01,   1.08639167e-01,
          5.53896451e-01],
       [  1.93370476e+00,   5.39236579e-01,   8.87910623e-02,
          1.70123446e+00,   3.30033576e+00,   1.52302282e-01,
          1.32743112e+00,   1.80722683e-01,   4.03131631e+00,
          1.90677251e-01],
       [  3.25159240e-02,   9.09431118e-02,   1.93258571e+00,
          3.18571840e-01,   2.55490023e+00,   7.41512897e-01,
          2.07084904e-01,   1.99951423e+00,   9.69941274e-01,
          1.96514773e+00],
       [  1.58104072e-01,   1.76582935e-01,   6.00264127e-02,
          1.84162010e-01,   2.31703470e+00,   5.97879925e-01,
          1.39618059e-01,   5.15432917e-01,   3.96470871e-01,
          6.25502177e-01],
       [  5.37171871e-01,   9.18961906e-03,   4.32993586e-01,
          4.81793086e-02,   3.86617063e-01,   1.40286173e+00,
          1.32182025e+00

In [18]:
test.signals()

array([[[[ -1.81263393e+00,  -1.74899708e+00,  -3.14119518e+00],
         [  1.33815296e+00,   5.20223226e-01,   1.25075410e+00],
         [  4.30600590e-01,  -4.80991616e-01,  -5.23727501e-01],
         [  2.04856280e+00,   1.02920403e+00,   1.91122889e+00]],

        [[ -1.65018735e-01,   1.15308427e+00,   4.83627874e+00],
         [ -1.80321097e-01,  -1.96301325e-01,   1.51483704e+00],
         [ -2.32095464e-01,  -2.63446122e+00,  -2.52833506e+00],
         [  2.83916841e+00,   4.99095446e+00,   5.56841639e+00]],

        [[  1.34081800e+00,  -8.39750595e+00,  -9.45794837e+00],
         [  2.21895266e+00,   9.41777349e+00,   5.39903257e+00],
         [  1.61249649e+01,   1.15823752e+01,   1.96942400e+01],
         [  7.40872052e+00,   1.10223033e+01,   2.46450230e+00]],

        [[  5.21270161e+00,   1.14536765e+01,   2.39166003e+01],
         [ -4.75038972e+00,  -5.48909474e+00,  -7.46924920e+00],
         [  2.68962546e-01,   1.04015414e+01,   1.90229755e+01],
         [  2.90566

In [19]:
test.cum_signals()

array([[[[  6.40245103e-01,   1.35537861e+00,   1.97544625e+00],
         [ -1.99797790e+00,  -3.83111374e+00,  -6.22318215e+00],
         [  2.58682245e-01,   6.52911310e-01,   7.08674817e-01],
         [  1.15097316e+00,   1.95655103e+00,   2.91886550e+00]],

        [[  4.90718956e-01,   1.64247875e+00,   2.03982084e+00],
         [ -1.64648237e+00,  -3.97200365e+00,  -6.23990671e+00],
         [  1.92819109e-01,   1.42657935e-01,   2.99502061e-01],
         [  7.82966130e-01,   2.20550542e+00,   3.89045426e+00]],

        [[ -3.73996363e-02,   3.58540655e-01,  -2.88089886e+00],
         [ -8.54894154e-02,  -2.14612364e+00,  -5.86639576e-01],
         [ -4.00590670e+00,  -2.87011118e+00,  -5.87399868e+00],
         [ -6.55085051e-01,  -2.73562619e+00,  -1.61633953e+00]],

        [[ -2.87746270e+00,  -5.58923225e+00,  -4.74447281e+00],
         [  5.05009522e+00,   3.10562606e+00,   4.57369252e+00],
         [ -5.25074303e+00,  -4.03189572e+00,  -7.52569254e+00],
         [  1.04880

In [20]:
test.expected_chi()

array([[[[  7.57431443e+01,   6.98163116e+01,   3.35015620e+01],
         [ -3.10811551e+02,  -1.80517919e+02,  -1.04524065e+02],
         [ -1.29709636e+02,  -1.52300752e+02,  -1.09270378e+02],
         [ -2.76953796e+02,   9.23511025e+00,   1.39381426e+01]],

        [[  9.59929159e+01,   3.75145936e+01,   2.17898454e+01],
         [ -2.11333688e+02,  -3.18430754e+01,  -6.66778245e+01],
         [  2.25669937e+01,   9.17550552e+00,  -1.38021688e+01],
         [  1.07375233e+02,  -1.40838750e+02,  -8.80935404e+01]],

        [[ -1.14445924e+00,  -1.19180313e+00,  -1.39180249e+00],
         [ -7.01607957e+00,  -1.56720640e+00,  -7.87620949e-01],
         [ -5.73799858e-01,  -1.76030425e+00,  -1.39227757e+00],
         [  3.99124712e+00,  -6.70808306e-01,  -2.21905723e-01]],

        [[ -4.84244836e-01,  -1.00334455e+00,  -8.20020790e-01],
         [  1.56859936e+00,   7.69608428e-01,   1.18006142e+00],
         [ -2.94797145e+00,  -1.63732111e+00,  -4.71128114e-01],
         [ -7.46695

In [21]:
test.means()

array([ -1.00675694e+02,   2.22681336e-01,   3.85557510e-01,
         5.22584655e-02,  -7.53677726e-01,   3.17445102e-01,
        -3.28625130e+01,  -5.51629206e-02,   7.29394345e-02,
         1.37874639e+00])

In [22]:
test.means(each_sim = False)

1.4000999864144357

In [23]:
test.variances()

array([  7.28678699e+00,   8.11911073e-01,   1.16184285e+02,
         4.52524969e+01,   1.22209171e+03,   1.65356166e+01,
         1.08777217e+00,   1.89064602e+01,   5.06492981e+01,
         2.34726830e+02])

In [24]:
test.variances(each_sim = False)

275070.65635813749